In [40]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [55]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
df_refugee = df_refugee[df_refugee['Year']>1984]
display(df_refugee.size)

1492205

809155

In [56]:
df_geo  = pd.read_csv("geo.csv",na_values ='NaN',keep_default_na =False)
display(df_geo.head())

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.93911,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [57]:
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['Year','latitude','longitude','country','Value','Origin']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_asylum','longitude': 'longitude_asylum','country':'country_asylum'})
display(df_merge.head(10))

,Year,latitude_asylum,longitude_asylum,country_asylum,Value,Origin
0,1985,-11.202692,17.873887,AO,13220.0,Dem. Rep. of the Congo
1,1985,-11.202692,17.873887,AO,69000.0,Namibia
2,1985,-11.202692,17.873887,AO,9300.0,South Africa
3,1986,-11.202692,17.873887,AO,13200.0,Dem. Rep. of the Congo
4,1986,-11.202692,17.873887,AO,69000.0,Namibia
5,1986,-11.202692,17.873887,AO,50.0,Various/Unknown
6,1986,-11.202692,17.873887,AO,10000.0,South Africa
7,1987,-11.202692,17.873887,AO,12210.0,Dem. Rep. of the Congo
8,1987,-11.202692,17.873887,AO,940.0,Dem. Rep. of the Congo
9,1987,-11.202692,17.873887,AO,69000.0,Namibia


In [58]:
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
display(df_merge.head())
df_merge = df_merge[['Year','country','latitude','longitude','country_asylum','latitude_asylum','longitude_asylum','Value']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_origin','longitude': 'longitude_origin','country':'country_origin'})
display(df_merge.head())
df_merge = df_merge.sort_values("Year")

,Year,latitude_asylum,longitude_asylum,country_asylum,Value,Origin,country,latitude,longitude,name
0,1985,-11.202692,17.873887,AO,69000.0,Namibia,NA,-22.95764,18.49041,Namibia
1,1986,-11.202692,17.873887,AO,69000.0,Namibia,NA,-22.95764,18.49041,Namibia
2,1987,-11.202692,17.873887,AO,69000.0,Namibia,NA,-22.95764,18.49041,Namibia
3,1988,-11.202692,17.873887,AO,69000.0,Namibia,NA,-22.95764,18.49041,Namibia
4,1989,-11.202692,17.873887,AO,1145.0,Namibia,NA,-22.95764,18.49041,Namibia


,Year,country_origin,latitude_origin,longitude_origin,country_asylum,latitude_asylum,longitude_asylum,Value
0,1985,NA,-22.95764,18.49041,AO,-11.202692,17.873887,69000.0
1,1986,NA,-22.95764,18.49041,AO,-11.202692,17.873887,69000.0
2,1987,NA,-22.95764,18.49041,AO,-11.202692,17.873887,69000.0
3,1988,NA,-22.95764,18.49041,AO,-11.202692,17.873887,69000.0
4,1989,NA,-22.95764,18.49041,AO,-11.202692,17.873887,1145.0


In [59]:
display(df_merge)

,Year,country_origin,latitude_origin,longitude_origin,country_asylum,latitude_asylum,longitude_asylum,Value
0,1985,NA,-22.95764,18.49041,AO,-11.202692,17.873887,69000.0
51693,1985,HT,18.971187,-72.285215,DO,18.735693,-70.162651,6000.0
50469,1985,ZW,-19.015438,29.154857,BW,-22.328474,24.684866,4250.0
1202,1985,ZA,-30.559482,22.937506,ZW,-19.015438,29.154857,250.0
1179,1985,ZA,-30.559482,22.937506,ZM,-13.133897,27.849332,3100.0
119503,1985,GQ,1.650801,10.267895,IT,41.87194,12.56738,1.0
46383,1985,ER,15.179384,39.782334,IT,41.87194,12.56738,185.0
46365,1985,ER,15.179384,39.782334,IQ,33.223191,43.679291,1200.0
23185,1985,SD,12.862807,30.217636,CF,6.611111,20.939444,850.0
23186,1985,SD,12.862807,30.217636,CF,6.611111,20.939444,400.0


In [46]:
df_merge.to_csv("final_data.csv", sep=',', encoding='utf-8',index = False)